In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class VanillaCGN(nn.Module):
    def __init__(self, input_dim, node_dim, n_layers) -> None:
        super().__init__()
        self.input_dim = input_dim
        self.node_dim = node_dim
        self.n_layers = n_layers
        self.U0 = nn.init.kaiming_normal_(torch.empty(input_dim, node_dim))
        self.b0 = nn.Parameter(torch.randn(node_dim))
        self.convLayers = nn.ModuleList(
            [ConvNetLayer(self.node_dim) for _ in range(self.n_layers)]
        )
        self.readOutLayer = GraphRegressionReadoutLayer(node_dim=node_dim)

    def forward(self, x, adj_mat):
        x = x @ self.U0 + self.b0  # self.b0 is broadcasted properly?
        for i in range(self.n_layers):
            x = self.convLayers[i](x, adj_mat)
        x = x.sum(dim=0)  # Check
        x = self.readOutLayer(x)
        return x


class ConvNetLayer(nn.Module):
    def __init__(self, node_dim) -> None:
        super().__init__()
        self.U = nn.Parameter(torch.randn(node_dim, node_dim))

    def forward(self, x, adj_mat):
        new_x = torch.empty_like(x)
        for i in range(x.shape[0]):
            deg_i = adj_mat[:, i].sum()
            mask_i = adj_mat[:, i] > 0
            new_x[i, :] = F.relu(
                self.U @ (x[mask_i, :].sum(dim=0)).to(torch.float32) / deg_i
            )
        return new_x


class GraphRegressionReadoutLayer(nn.Module):
    def __init__(self, node_dim) -> None:
        super().__init__()
        self.node_dim = node_dim
        self.Q = nn.Parameter(nn.init.kaiming_normal_(torch.empty(node_dim, node_dim)))
        self.P = nn.Parameter(nn.init.kaiming_normal_(torch.empty((1, node_dim))))

    def forward(self, x):
        return (self.P @ F.relu(self.Q @ x)).squeeze()

def build_adj_mat(edge_index):
    """Build the adjacency matrix of a graph from edge_index that looks like this: [[1, 4, 5, 6], [2, 3, 3, 5]]
    cf. https://huggingface.co/datasets/graphs-datasets/ZINC for more details"""
    n_nodes = max(max(edge_index[0]), max(edge_index[1])) + 1
    n_edges = len(edge_index[0])
    adj_mat = torch.zeros((n_nodes, n_nodes))
    for i in range(n_edges):
        adj_mat[edge_index[0][i], edge_index[1][i]] = 1
    adj_mat += torch.eye(adj_mat.shape[0])  # Ensures deg_i > 0 and stabilize training
    return adj_mat

In [ ]:
model = VanillaCGN(input_dim=1, node_dim=10, n_layers=4)
y_pred = model(node_feat, adj_mat)

# Y = torch.rand((3, 2))
# edge_index = [[0, 1, 1, 2],[1, 0, 2, 1]]
# adj_mat = build_adj_mat(edge_index)
# # for i in range(5):
# Y = model(Y, adj_mat)
# print(Y)

In [72]:
node_feat = torch.tensor([[[0],
         [1],
         [0],
         [0],
         [0],
         [1],
         [0],
         [2],
         [0],
         [1],
         [4],
         [0],
         [0],
         [1],
         [2],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0]]])

adj_mat = torch.tensor([[[1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
          0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
          0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
          1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          1., 1., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 1., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 1.]]])

In [28]:
# adj_mat = torch.tensor([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
# X = torch.tensor([[5, 2], [3, 4], [10, 20]], dtype=torch.float32)
# layer = ConvNetLayer(node_dim=2)

# X = layer(X)
# print(X)

TypeError: ConvNetLayer.forward() missing 1 required positional argument: 'adj_mat'